In [12]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement seaborn (from versions: none)
ERROR: No matching distribution found for seaborn


In [2]:
import torch
import os
import math
import json

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import numpy as np
import pandas as pd
from tqdm import tqdm
from time import sleep
#import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from urllib.request import urlopen

%matplotlib inline
%config InlineBackend.figure_format='retina'
# sns.set(style='whitegrid', palette='muted', font_scale=1.2)
# HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]
# sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 17, 10
register_matplotlib_converters()
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [3]:
def create_sequences(data, deviation, seq_length=30):
    xs = []
    ys = []
    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        dev = deviation[i+seq_length]
        xs.append(x)
        ys.append([y,dev])
    return np.array(xs).reshape(-1,seq_length), np.array(ys).reshape(-1,2)

In [4]:
def smooth_it(data):
    new_data=[]
    deviation=[]
    i=0
    while(i<len(data)):
        sum=0
        for j in range(-6,1):
            if(i+j>=0 and i+j<len(data)):            
                sum+=data[i+j]
            else:
                sum+=data[i]
        new_data.append(sum/7)
        deviation.append(abs(new_data[i]-data[i]))
        i+=1
    return new_data, deviation

In [5]:
def daily_cases(data_json):
  data=[]
  sz = len(data_json['data']['timeline'])
  for i in reversed(range(sz)):
    DailyConfirmed = data_json['data']['timeline'][i]['new_confirmed']
    data.append(DailyConfirmed)
  new_data, dev = smooth_it(data)
  mx=max(new_data, default=0)
  data_scaled = [(x/mx)*1000 for x in new_data]
  return data_scaled,mx,dev

In [6]:
# # reading ISO codes of countries
# url_codes = "https://www.nationsonline.org/oneworld/country_code_list.htm"

# #html = requests.get(url_codes).content
# session = requests.Session()
# retry = Retry(connect=3, backoff_factor=0.5)
# adapter = HTTPAdapter(max_retries=retry)
# session.mount('http://', adapter)
# session.mount('https://', adapter)
# html = session.get(url_codes)
# print('success')
# df_list = pd.read_html(html)
X_all = np.empty((0,30))
y_all = np.empty((0,2))
# for idx in range(len(df_list)):
#   for ISO in df_list[idx][2]:

response_ISO = urlopen("https://pkgstore.datahub.io/core/country-list/data_json/data/8c458f2d15d9f2119654b29ede6e45b8/data_json.json")
dict_ISO = json.loads(response_ISO.read())
for idx in range(len(dict_ISO)):
    ISO = dict_ISO[idx]['Code']
    try:
      response = urlopen(f"https://corona-api.com/countries/{ISO}")
    except:
      continue
    data_json = json.loads(response.read())
    data, mx, dev = daily_cases(data_json)
    X_train, y_train = create_sequences(data, dev)
    X_all = np.concatenate((X_all, X_train))
    y_all = np.concatenate((y_all, y_train))
print(X_all.shape, y_all.shape)
    

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [ ]:
X_all = torch.from_numpy(X_all).float()
y_all = torch.from_numpy(y_all).float()

In [ ]:
class getData(Dataset):
   def __init__(self, data, targets):
        self.data = torch.tensor(data)
        self.targets = torch.tensor(targets)
        
   def __len__(self):
        return len(self.data)

   def __getitem__(self, index):
        target = self.targets[index]
        data_values = self.data[index]
        return data_values.float(),target.float()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
batch_size = 128
train_dataset = getData(X_all, y_all)
kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
class CoronaVirusPredictor(nn.Module):
  def __init__(self, n_features, n_hidden, seq_len, n_layers=2):
    super(CoronaVirusPredictor, self).__init__()
    self.n_hidden = n_hidden
    self.seq_len = seq_len
    self.n_layers = n_layers
    self.lstm = nn.LSTM(
      input_size=n_features,
      hidden_size=n_hidden,
      num_layers=n_layers,
      batch_first=True,
      #dropout=0.2
    )
    self.dropout = nn.Dropout(0.2)
    self.linear = nn.Linear(in_features=n_hidden, out_features=32)
    self.linear2 = nn.Linear(32,2)
    #self.relu = nn.ReLU()

  def forward(self, sequences):
    lstm_out, (h_out, _) = self.lstm(
      sequences.view(len(sequences), self.seq_len, -1)
    )
    #h_out = h_out.view(-1, self.n_hidden) #only work for single layer
    out = lstm_out[:,-1,:].view(-1, self.n_hidden)
    out = self.dropout(out)
    out = self.linear(out)
    out = self.linear2(out)
    #out = self.relu(out)
    return out

In [ ]:
def train_model(model, train_loader, X_test, y_test, num_epochs, initial_lr=0.001, step_size=200, gamma=0.5):
  loss_fn = torch.nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
  num_epochs = num_epochs
  model = model.to(device)

  num_batches = math.floor(X_all.shape[0]/batch_size)
  for t in range(num_epochs):
    running_loss = 0
    for _, data in enumerate(train_loader, 0):
      input_seq,target = data
      #print(input_seq.shape, target.shape)
      input_seq = input_seq.to(device)
      target = target.to(device)
      y_pred = model(input_seq)
      loss = loss_fn(y_pred.float(), target)
      running_loss += loss.item()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    scheduler.step()
    
    if X_test is not None:
      with torch.no_grad():
        X_test = X_test.to(device)
        y_test = y_test.to(device)
        y_test_pred = model(X_test)
        test_loss = loss_fn(y_test_pred.float(), y_test)
  
      if t % 50 == 0:
        print(f'Epoch {t} train loss: {running_loss/num_batches}  AND test loss: {test_loss.item()}')

    elif t % 10 == 0:
        print(f'Epoch {t} train loss: {running_loss/num_batches}')
    
  return model.eval()

In [ ]:
model = CoronaVirusPredictor(
  n_features=1,
  n_hidden=64,
  seq_len=30,
  n_layers=4
)
model

In [ ]:
model = train_model(
    model,
    train_loader,
    None,
    None,
    800,
    initial_lr=0.01,
    step_size=50,
    gamma=0.5
)
torch.save(model.state_dict(),'/workspace/LSTM_all_countries.pth')

In [ ]:
response = urlopen("https://corona-api.com/countries/IN")
data_json_IN = json.loads(response.read())
data_IN,mx_IN = daily_cases(data_json_IN)
X_IN, y_IN = create_sequences(data_IN)
X_IN = torch.from_numpy(X_IN).float()
y_IN = torch.from_numpy(y_IN).float()

In [ ]:
DAYS_TO_PREDICT = 200
seq_length = 30
with torch.no_grad():
  test_seq = X_IN[-1:]
  preds_IN = []
  for _ in range(DAYS_TO_PREDICT):
    #print(test_seq)
    test_seq = torch.tensor(test_seq).float().to(device)
    y_test_pred = model(test_seq)
    pred = torch.flatten(y_test_pred).item()
    preds_IN.append(pred)
    new_seq = test_seq.cpu().numpy().flatten()
    new_seq = np.append(new_seq, [pred])
    new_seq = new_seq[1:]
    test_seq = torch.as_tensor(new_seq).view(1, seq_length, 1).float()

In [ ]:
preds_IN_final = [(x*mx_IN)/1000 for x in preds_IN]
data_IN_final = [(x*mx_IN)/1000 for x in data_IN]

In [ ]:
initial = 400
DAYS_TO_PREDICT = len(data_IN_final)-initial
seq_length = 30
with torch.no_grad():
  test_seq = X_IN[initial:initial+1]
  preds_IN_2nd = []
  for _ in range(DAYS_TO_PREDICT):
    #print(test_seq)
    test_seq = torch.tensor(test_seq).float().to(device)
    y_test_pred = model(test_seq)
    pred = torch.flatten(y_test_pred).item()
    preds_IN_2nd.append(pred)
    new_seq = test_seq.cpu().numpy().flatten()
    new_seq = np.append(new_seq, [pred])
    new_seq = new_seq[1:]
    test_seq = torch.as_tensor(new_seq).view(1, seq_length, 1).float()

In [ ]:
preds_IN_final_2nd = [(x*mx_IN)/1000 for x in preds_IN_2nd]

In [ ]:
predicted_cases_3rd = pd.Series(
  data=preds_IN_final,
  index=[len(data_IN_final)+i for i in range(len(preds_IN_final))]
)
predicted_cases_2nd = pd.Series(
  data=preds_IN_final_2nd,
  index=[i+seq_length for i in range(initial,len(data_IN_final))]
)

In [ ]:
plt.plot(data_IN_final, label='Historical Daily Cases')
plt.plot(predicted_cases, label='Predicted Daily Cases for 200 days')
#plt.plot(predicted_cases_2nd, label='2nd wave prediction')
plt.legend();